# Sparse Document Retriever - Implantação

Preencha aqui com detalhes sobre a tarefa.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np
import json
from aux_functions import build_result_dataframe


class Model:
    
    def __init__(self):
        self.loaded = False
    
    def load(self):
        
        artifacts = joblib.load("/tmp/data/sparse_retriever.joblib")
        self.model = artifacts["model"]
        self.top = artifacts["top"]
        self.question = artifacts["question"]
        self.columns = artifacts["columns"]
        self.column_context = artifacts["column_context"]
        self.column_doc_id = artifacts["column_doc_id"]
        self.column_score = artifacts["column_score"]
        self.loaded = True

    def predict(self, X, feature_names, meta=None):
        
        if not self.loaded:
            self.load()
            
        if not feature_names:
            feature_names = self.columns
            
        
        df = pd.DataFrame(X,columns = feature_names)
        report_contexts = df[self.column_context].to_numpy()
        doc_ids = df[self.column_doc_id].to_numpy()
        self.model.fit(contexts=report_contexts, doc_ids=doc_ids)
        sim_contexts_ids, scores = self.model(questions=self.question, top=self.top)

            
        df_result = build_result_dataframe(df_input=df,
                            sim_contexts_ids=sim_contexts_ids[0],
                            scores=scores[0],
                            column_doc_id = self.column_doc_id,
                            column_score = self.column_score)
        
        return df_result.to_numpy() 

Overwriting Model.py


In [4]:
#import pandas as pd
#df = pd.read_csv("/tmp/data/expanded_en-6.csv")
#X = df.to_numpy()

In [5]:
# from Model import Model
# model = Model()
# result = model.predict(X,None)
# result